### QFT SIZE 8,16,32 IN IBM (Falcon) AND QUANTINUUM

In [1]:
# Caller
from pathlib import Path
import sys

# Discover project root
proj = Path.cwd().resolve()
for _ in range(8):
    if (proj / "results" / "pairwise" / "summary").exists() and (proj / "notebooks").exists():
        break
    if proj.parent == proj:
        break
    proj = proj.parent

# Import helper
sys.path.insert(0, str(proj / "notebooks"))
from notebook_helper import load_family, tidy, show_table  # (sort_best_worst, add_rank_within_size available if needed)

# analysis parameters
FAMILY = "qft"  # or: "qaoa","vqe_two_local","qft","grover","vbe_adder","randomcircuit"
BACKENDS = ("ibm_falcon","quantinuum")

loaded = load_family(FAMILY, BACKENDS)

for backend, df in loaded.items():
    tbl = tidy(df)
    show_table(tbl, caption=f"{FAMILY.upper()} – {backend} (raw tidy)")

    # To rank later, uncomment:
    # from notebook_helper import sort_best_worst, add_rank_within_size
    # PRIMARY_METRIC = "depth_delta"  # or "n_ops_total_gates_delta", "n_ops_total_delta", etc.
    # ranked = sort_best_worst(tbl, primary=PRIMARY_METRIC, per_size=True)
    # ranked = add_rank_within_size(ranked, by=PRIMARY_METRIC)
    # show_table(ranked, caption=f"{FAMILY.upper()} – {backend} (ranked by {PRIMARY_METRIC})")

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,ibm_falcon,qft,8,-,RB→RR,63,61,-2,185,170,-15,176,162,-14,116,102,-14,68,68,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 86], [""cx"", 68], [""sx"", 8], [""measure"", 8]]",8,8,2025-10-05T22:23:33
1,ibm_falcon,qft,8,-,RR→RB,63,61,-2,185,170,-15,176,162,-14,116,102,-14,68,68,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 86], [""cx"", 68], [""sx"", 8], [""measure"", 8]]",8,8,2025-10-05T22:23:33
2,ibm_falcon,qft,8,-,RB→CTM,63,84,+21,185,184,-1,176,176,+0,116,116,+0,68,68,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8]]",8,8,2025-10-05T22:23:33
3,ibm_falcon,qft,8,-,CTM→RB,63,84,+21,185,184,-1,176,176,+0,116,116,+0,68,68,+0,1,0,-1,1,0,-1,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8]]",8,8,2025-10-05T22:23:33
4,ibm_falcon,qft,8,-,RR→CTM,63,71,+8,185,171,-14,176,162,-14,116,102,-14,68,68,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""rz"", 86], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-05T22:23:33
5,ibm_falcon,qft,8,-,CTM→RR,63,55,-8,185,150,-35,176,141,-35,116,81,-35,68,68,+0,1,1,+0,1,1,+0,8,8,+0,0,0,+0,"[[""rz"", 100], [""cx"", 68], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]","[[""cx"", 68], [""rz"", 65], [""sx"", 8], [""measure"", 8], [""barrier"", 1]]",8,8,2025-10-05T22:23:33
6,ibm_falcon,qft,16,-,RB→RR,127,125,-2,689,658,-31,672,642,-30,424,394,-30,264,264,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 362], [""cx"", 264], [""sx"", 16], [""measure"", 16]]",16,16,2025-10-05T22:23:33
7,ibm_falcon,qft,16,-,RR→RB,127,125,-2,689,658,-31,672,642,-30,424,394,-30,264,264,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 362], [""cx"", 264], [""sx"", 16], [""measure"", 16]]",16,16,2025-10-05T22:23:34
8,ibm_falcon,qft,16,-,RB→CTM,127,232,+105,689,688,-1,672,672,+0,424,424,+0,264,264,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16]]",16,16,2025-10-05T22:23:34
9,ibm_falcon,qft,16,-,CTM→RB,127,232,+105,689,688,-1,672,672,+0,424,424,+0,264,264,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 392], [""cx"", 264], [""sx"", 16], [""measure"", 16]]",16,16,2025-10-05T22:23:34


,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,quantinuum,qft,32,-,RB→RR,139,138,-1,1921,1856,-65,1888,1824,-64,1376,1312,-64,544,544,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""rz"", 1024], [""zzphase"", 544], [""ry"", 128], [""rx"", 128], [""measure"", 32]]",32,32,2025-10-05T22:23:35
1,quantinuum,qft,32,-,RR→RB,139,138,-1,1921,1872,-49,1888,1840,-48,1376,1328,-48,544,544,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""rz"", 1040], [""zzphase"", 544], [""ry"", 128], [""rx"", 128], [""measure"", 32]]",32,32,2025-10-05T22:23:35
2,quantinuum,qft,32,-,RB→CTM,139,622,+483,1921,1920,-1,1888,1888,+0,1376,1376,+0,544,544,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32]]",32,32,2025-10-05T22:23:35
3,quantinuum,qft,32,-,CTM→RB,139,622,+483,1921,1920,-1,1888,1888,+0,1376,1376,+0,544,544,+0,1,0,-1,1,0,-1,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32]]",32,32,2025-10-05T22:23:35
4,quantinuum,qft,32,-,RR→CTM,139,619,+480,1921,1873,-48,1888,1840,-48,1376,1328,-48,544,544,+0,1,1,+0,1,1,+0,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""rz"", 1040], [""zzphase"", 544], [""ry"", 128], [""rx"", 128], [""measure"", 32], [""barrier"", 1]]",32,32,2025-10-05T22:23:36
5,quantinuum,qft,32,-,CTM→RR,139,139,+0,1921,928,-993,1888,895,-993,1376,383,-993,544,544,+0,1,1,+0,1,1,+0,32,32,+0,0,0,+0,"[[""rz"", 1040], [""zzphase"", 544], [""rx"", 176], [""ry"", 128], [""measure"", 32], [""barrier"", 1]]","[[""zzphase"", 544], [""ry"", 128], [""rx"", 128], [""rz"", 95], [""measure"", 32], [""barrier"", 1]]",32,32,2025-10-05T22:23:36
6,quantinuum,qft,16,-,RB→RR,75,74,-1,577,544,-33,560,528,-32,432,400,-32,144,144,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 256], [""zzphase"", 144], [""ry"", 64], [""rx"", 64], [""measure"", 16]]",16,16,2025-10-05T22:23:36
7,quantinuum,qft,16,-,RR→RB,75,74,-1,577,552,-25,560,536,-24,432,408,-24,144,144,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 264], [""zzphase"", 144], [""ry"", 64], [""rx"", 64], [""measure"", 16]]",16,16,2025-10-05T22:23:36
8,quantinuum,qft,16,-,RB→CTM,75,190,+115,577,576,-1,560,560,+0,432,432,+0,144,144,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16]]",16,16,2025-10-05T22:23:36
9,quantinuum,qft,16,-,CTM→RB,75,190,+115,577,576,-1,560,560,+0,432,432,+0,144,144,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16], [""barrier"", 1]]","[[""rz"", 264], [""zzphase"", 144], [""rx"", 88], [""ry"", 64], [""measure"", 16]]",16,16,2025-10-05T22:23:36


### quick findings

- **RB ↔ RR:** small but consistent depth reductions (IBM: −2, QNU: −1) barriers removed, 2 qubit gates unchanged, savings driven by fewer 1-qubit ops
- **RB ↔ CTM:** depth increases (size-dependent, small → very large), barriers removed, little change to gates 
- **RR ↔ CTM:** directional effect : **CTM → RR** often reduces depth (and cuts ops / 1-qubit gates) while **RR → CTM** can increase depth (magnitude varies by size) barrier remains present
- Magnitude differences between IBM and Quantinuum are primarily in how much depth and 1q changes. the sign/patterns are consistent.